In [91]:
import math
import matplotlib
import matplotlib.pyplot as plt
import random

In [152]:
#Calculando quantos bits são necessários para representar qualquer valor no dominio de -100 a 100
#Precisamos ajustar(arredondar) a quantidade de bits para que seja inteiro
def bits_necessarios(dominio_max, dominio_min, precisao_casas_decimais):
    k_bits = math.ceil(math.log2((dominio_max - dominio_min) * (10 ** precisao_casas_decimais)))
    return k_bits

In [153]:
#Definindo a precisão
def precisao(dominio_max,dominio_min,bits):
    precisao = (dominio_max - dominio_min) / ((2 ** bits) -1)
    return precisao

In [281]:
def funcaoObjetiva(x, y):
    inner = (x ** 2) + (y ** 2)
    sin_inner = math.sin(math.sqrt(inner * 10000))  # Multiplicação por 10000 para compensar a escala de [0, 1]
    numerator = (sin_inner ** 2) - 0.5
    denominator = 1 + 000.1 * inner
    f6 = 0.5 - (numerator / (denominator ** 2))
    return round(f6, 10)

In [199]:
#Criando uma População Inicial Aleatória 50 bits 25 para x e 25 para y
def gerarPopulacao(individuos):
    populacao = []
    cont = 0
    while cont < individuos:
        caracteres = ""
        for cromossomo in range(0, 50):
            caracteres = caracteres + str(random.randint(0, 1))
        populacao.append(caracteres)
        cont = cont + 1
    return populacao

In [200]:
p = gerarPopulacao(10)

In [201]:
def geraPopulacaoSeparada(populacao):
    cadeia_gene_separada = []
    for i in range(0, len(populacao)):
        x = populacao[i][:25]
        y = populacao[i][25:]
        cadeia_gene_seperada_eixos = [x, y]
        cadeia_gene_separada.append(cadeia_gene_seperada_eixos)
    return cadeia_gene_separada

In [209]:
## Criando o Decodificador para representao o valor em binário para a representação no domínio em forma real
def decodificador_decimal(cadeia_gene_separada):
    decimais = []
    for i in range(0, len(cadeia_gene_separada)):
        v1 = int(cadeia_gene_separada[i][0], 2)
        v2 = int(cadeia_gene_separada[i][1], 2)
        decimais.append([v1,v2])
    return decimais

In [267]:
## Realizaremos agora a conversão do objeto decimal, para o que é representado no domínio
def decodificador_real(decodificados_decimais, dominio_max, dominio_min, precisao_dominio):
    decodificados_reais = []
    for i in range(len(decodificados_decimais)):
        v1 = round(dominio_min + precisao_dominio * decodificados_decimais[i][0],5)
        v2 = round(dominio_min + precisao_dominio * decodificados_decimais[i][1],5)
        decodificados_reais.append([v1, v2])
    return decodificados_reais

In [277]:
def ajustar_escala(valores_reais, dominio_max, dominio_min):
    diferenca = dominio_max - dominio_min
    razao = 1 / diferenca
    valores_ajustados = []
    for par in valores_reais:
        valor_ajustado_x = (par[0] - dominio_min) * razao
        valor_ajustado_y = (par[1] - dominio_min) * razao
        valores_ajustados.append([valor_ajustado_x, valor_ajustado_y])
    return valores_ajustados

In [278]:
#Vamos aplicar a decodificação dos valores binários para os valores reais para podermos futuramente calcular o fitnnes
def decodificarPopulacao(populacao,dominio_max,dominio_min,precisao_casas_decimais):
    bits = bits_necessarios(dominio_max, dominio_min, precisao_casas_decimais)
    precisao_dominio = precisao(dominio_max,dominio_min,bits)
    cadeia_gene_separada = geraPopulacaoSeparada(populacao)
    dec_decimais = decodificador_decimal(cadeia_gene_separada)
    dec_reais = decodificador_real(dec_decimais,dominio_max,dominio_min,precisao_dominio)
   # Supondo que 'decodificados_reais' seja a lista de valores reais decodificados
    # de sua população
    decodificados_reais_ajustados = ajustar_escala(dec_reais, dominio_max,dominio_min)
    return decodificados_reais_ajustados

In [282]:
decodificarPopulacao(p,100,-100,5)

[[0.2263452, 0.65729495],
 [0.658831, 0.045310699999999995],
 [0.6084854000000001, 0.4310139],
 [0.2712315, 0.90331035],
 [0.5470991500000001, 0.49688585],
 [0.6900833000000001, 0.9944095999999999],
 [0.35847615, 0.5460504],
 [0.38040725000000003, 0.11652295000000003],
 [0.9832132, 0.4050391],
 [0.88815465, 0.12944405000000003]]

In [322]:
funcaoObjetiva(0.2263452,0.65729495)

0.8153409475

In [288]:
def calcular_aptidoes(populacao_decodificada):
    aptidoes = []
    for individuo in populacao_decodificada:
        x, y = individuo
        aptidao = funcaoObjetiva(x, y)
        aptidoes.append(aptidao)
    return aptidoes

In [289]:
def particaoPopulacao(aptidoes):
    proporcoes = []
    soma_aptidoes = sum(aptidoes)
    for aptidao in aptidoes:
        proporcao_individuo_atual = aptidao / soma_aptidoes
        proporcoes.append(proporcao_individuo_atual)
    return proporcoes

In [290]:
def criandoRoleta(probabilidades_relativas):
    roleta = []
    acumulado = 0
    for probabilidade in probabilidades_relativas:
        acumulado += probabilidade
        roleta.append(acumulado)
    return roleta

In [291]:
##def girandoRoleta(roleta,num_selecionados):
def girandoRoleta(roleta, quantidade_individuos):
    individuos_selecionados = []
    for i in range(0,quantidade_individuos):
        ##Giro a roleta
        giro_aleatorio = random.random()
        #Selecionando ddeposi que a roleta gira
        for i in range(len(roleta)):
            if roleta[i] >= giro_aleatorio:
                individuos_selecionados.append(i)
                break
    return individuos_selecionados

In [292]:
def reproducao(individuos_selecionados,taxa_cruzamento,populacao):
    ###### Defino a taxa de cruzamento para os pai ######
    taxa_primeira_parte_do_filho = round((taxa_cruzamento * 50) / 100) ## 50 quantidade de bits
    taxa_segunda_parte_do_filho = 50 - taxa_primeira_parte_do_filho ## Quantos bits serão usados no segundo corte
    ###### Cria e Armazeno os individuos selecionados para ser usado no laço ######
    pais = []
    pais.append(populacao[individuos_selecionados[0]]) #Pai_01
    pais.append(populacao[individuos_selecionados[1]])  #Pai_02
    pontos_corte = []
    #Pont0s de corte de cada pai, para seus filhos
    ponto_corte_pai01 = random.randint(0,50) #Para fins de teste escolha 10 E 40
    ponto_corte_pai02 = random.randint(0,50) #Para fins de teste escolha 10 E 40
    # Mostra os pontos de Corte de cada pai
    ##print("Pai1: {} \nPai2 :{}".format(ponto_corte_pai01,ponto_corte_pai02))
    #print("Pai 1 {} \nPai                                           {}".format(pai_01,pai_01[40:]))
    #print("Pai 2 {} \nPai             {}".format(pai_02,pai_02[10:]))
    ###### Armazeno os ponto de corte para ser usado no laço ######
    pontos_corte.append(ponto_corte_pai01)
    pontos_corte.append(ponto_corte_pai02)
    ###### Cria um lista contendo os genes dos pais separados, dado o ponto de corte que eels receberam ######
    filhos_nao_cruzados = []
    ###### Indice de acesso a minha lista de pontos de corte ######
    ponto = 0
    ###### Atribuo a partir do ponto de corte de cada um dos pais, seus respectivos cromossomos os filho nãa cruzados ######
    for pai in range(0,len(pais)):
        #print("############################################# {} CICLO #############################################".format(pai))
        #print("############################################# Ponto de Corte {} = {}  ##############################".format(ponto,pontos_corte[ponto]))
        ## 38 Primeiro + 12 Primeiro -> 38 Segundo + 12 Segundo ....... 
        ######################################  38 Primeiro ##########################################################################################
        ## Para o primeiro Filho - Genes do Pai 01 
        #Se de onde eu começo bater até o limite, tá ok ainda, posso pegar so os bits do vetor mesmo
        if len(pais[pai][pontos_corte[ponto]:]) >= taxa_primeira_parte_do_filho: # Se Do indice que eu to até o fim for maior que meu limite de 38 bits 75%
            parte_01_filho = pais[pai][pontos_corte[ponto]:taxa_primeira_parte_do_filho + pontos_corte[ponto]] # Meu filho pega só os 37 que lhe cabem
            ##PRINTS ABAIXO APENAS PARA MOSTRAR OS CASOS EM QUE SÃO 10 E 40
            #print("Filho Direto")
            #print("Pai {}           {} \nPT1P1                     {}".format(pai,pais[pai],pais[pai][pontos_corte[ponto]:]))
            #print("Filho01                   {}".format(parte_01_filho))
            #print(f"Primeira parte do filho/Bits " + str(len(parte_01_filho)))
        else:
            #print("Filho particionado")
            parte_01_pai = pais[pai][pontos_corte[ponto]:]
            parte_02_pai = pais[pai][:taxa_primeira_parte_do_filho - len(pais[pai][pontos_corte[ponto]:])]
            parte_01_filho = parte_01_pai + parte_02_pai
            ##PRINTS ABAIXO APENAS PARA MOSTRAR OS CASOS EM QUE SÃO 10 E 40
            #print("Filho particionado")
            #print("Pai {}          {}".format(pai,pais[pai]))
            #print("PT1P1                                                  {}".format(parte_01_pai))
            #print("PT2P2          {}".format(parte_02_pai))
            #print("Filho          {}".format(parte_01_filho))
            #print(f"Primeira parte do Filho/Bits " + str(len(parte_01_filho)))
        ######################################  12 Primeiro ##########################################################################################
        ## Para o primeiro Filho - Genes do Pai 01
        #Se de onde eu começo bater até o limite, tá ok ainda, posso pegar so os bits do vetor mesmo
        if len(pais[pai][pontos_corte[ponto]:]) >= taxa_segunda_parte_do_filho: # Se Do indice que eu to até o fim for maior que meu limite de 38 bits 75%
            parte_02_filho = pais[pai][pontos_corte[ponto]:taxa_segunda_parte_do_filho + pontos_corte[ponto]] # Meu filho pega só os 37 que lhe cabem
            ##PRINTS ABAIXO APENAS PARA MOSTRAR OS CASOS EM QUE SÃO 10 E 40
            #print("Filho Direto")
            #print("Pai {}           {} \nPT1P1                     {}".format(pai,pais[pai],pais[pai][pontos_corte[ponto]:]))
            #print("Filho01                   {}".format(parte_02_filho))
            #print(f"Primeira parte do filho/Bits " + str(len(parte_02_filho)))
        else:
            #print("Filho particionado")
            parte_01_pai = pais[pai][pontos_corte[ponto]:]
            parte_02_pai = pais[pai][:taxa_segunda_parte_do_filho - len(pais[pai][pontos_corte[ponto]:])]
            parte_02_filho = parte_01_pai + parte_02_pai
            ##PRINTS ABAIXO APENAS PARA MOSTRAR OS CASOS EM QUE SÃO 10 E 40
            #print("Filho particionado")
            #print("Pai {}          {}".format(pai,pais[pai]))
            #print("PT1P1                                                  {}".format(parte_01_pai))
            #print("PT2P2          {}".format(parte_02_pai))
            #print("Filho          {}".format(parte_02_filho))
            #print(f"Primeira parte do Filho/Bits " + str(len(parte_02_filho)))
        filhos_nao_cruzados.append([parte_01_filho,parte_02_filho])    
        ponto = ponto + 1
    return filhos_nao_cruzados

In [293]:
def crossover(filhos_sem_cruzamento,populacao,individuos_selecionados):
    ###### Cria e Armazeno os individuos selecionados para ser usado no laço ######
    pais = []
    pais.append(populacao[individuos_selecionados[0]]) #Pai_01
    pais.append(populacao[individuos_selecionados[1]])  #Pai_02
    filhos_cruzados = []
    filho01 = filhos_sem_cruzamento[0][0] + filhos_sem_cruzamento[1][1]
    filho02 = filhos_sem_cruzamento[1][0] + filhos_sem_cruzamento[0][1]
    filhos_cruzados.append(filho01)
    filhos_cruzados.append(filho02)
    if filhos_cruzados[0] != pais[0] and filhos_cruzados[0] != pais[1] and filhos_cruzados[1] != pais[0] and filhos_cruzados[1] != pais[1] :
        return filhos_cruzados
       #print("ok")
    else:
        #print("Fodeu")
        novos_individuos_selecionados = girandoRoleta(roleta,2)
        nova_reproducao = reproducao(novos_individuos_selecionados,taxa_cruzamento)
        return crossover(nova_reproducao)

In [294]:
def mutacao(filhos_cruzados, taxa_mutacao):
    filhos_mutados = []
    for filho in filhos_cruzados:
        filho_mutado = ""
        for bit in filho:
            if random.random() < taxa_mutacao:
                # Com uma certa probabilidade (taxa_mutacao), o bit será invertido
                filho_mutado += "1" if bit == "0" else "0"
            else:
                filho_mutado += bit
        filhos_mutados.append(filho_mutado)
    return filhos_mutados

In [239]:
def trocaPaiseFilhos(populacao,filhos_mutados,aptidoes,dominio_max,dominio_min,precisao_dominio):
#lista binaria dos pais
#lista binaria dos filhos
#lista aptidão sem substituição
#2 piores indices dentro dessa lista sem substituição
#verifico se o piorpai é menor que o filho
#    se for faço a troca entre a lista binária dos pais, no 1 pior indice pelo 1 filho binario
#verifico se o segundo pior pai é menor que o segundo filho
#    se for faço a troca entre a lista binária dos pais, no 2 pior indice pelo 2 filho binario
    clonepop = populacao.copy()
    populacao_separada_filhos = geraPopulacaoSeparada(filhos_mutados) ## Etapa para avaliacao
    populacao_decodificada_filhos = decodificarPopulacao(populacao_separada_filhos,dominio_max,dominio_min,precisao_dominio) ## Etapa para avaliacao
    aptidoes_filhos_mutados = calculandoAptidao(populacao_decodificada_filhos)
    piores_indices = sorted(range(len(aptidoes)), key=lambda i: aptidoes[i])[:2]
    print(piores_indices)
    if aptidoes[piores_indices[0]] < aptidoes_filhos_mutados[0]:
        clonepop[piores_indices[0]] = filhos_mutados[0]
    if aptidoes[piores_indices[1]] < aptidoes_filhos_mutados[1]:
        clonepop[piores_indices[1]] = filhos_mutados[1]
    if clonepop != populacao:
        #print("Tudo Certo")
        #print("Indices que foram trocados")
        #print(piores_indices)
        return clonepop
    else:
        ## Seleção de indivíduos para reprodução ##
        individuos_selecionados = girandoRoleta(roleta, 2)
    
        ## Reprodução (Cruzamento dos selecionados) ##
        filhos_sem_cruzamento = reproducao(individuos_selecionados,taxa_de_cruzamento,populacao)
        filhos_cruzados = crossover(filhos_sem_cruzamento,populacao,individuos_selecionados)
   
        ## Mutação ##
        filhos_mutados = mutacao(filhos_cruzados, taxa_mutacao)
    return trocaPaiseFilhos(populacao,filhos_mutados,aptidoes,dominio_max,dominio_min,precisao_dominio)

In [111]:
t = 0
individuos = 100
NGEN = 100
taxa_mutacao = 0.01  #Taxa de mutação de 1%
taxa_de_cruzamento = 75 ## 75% 
precisao_casas_decimais = 5 ## Precisão de casas decimais
dominio_max = 100
dominio_min = -100

bits = bits_necessarios(dominio_max, dominio_min, precisao_casas_decimais)
precisao_dominio = precisao(dominio_max, dominio_min, bits)

populacao = gerarPopulacao(individuos)
populacao_decodificada = decodificarPopulacao(populacao, dominio_max, dominio_min, precisao_dominio)

aptidoes = calculandoAptidao(populacao_decodificada)
aptidoes_relativas = particaoPopulacao(aptidoes)

contador = 0
aptidao_alvo = 1.0
while contador < NGEN:
    contador = contador + 1
    if max(aptidoes) == aptidao_alvo:
        print(f'Solução ótima alcançada na geração {contador}')
        break
    roleta = criandoRoleta(aptidoes_relativas)
    individuos_selecionados = girandoRoleta(roleta, 2)
    filhos_sem_cruzamento = reproducao(individuos_selecionados, taxa_de_cruzamento, populacao)
    filhos_cruzados = crossover(filhos_sem_cruzamento, populacao, individuos_selecionados)
    filhos_mutados = mutacao(filhos_cruzados, taxa_mutacao)
    populacao = trocaPaiseFilhos(populacao, filhos_mutados, aptidoes, dominio_max, dominio_min, precisao_dominio)
    populacao_decodificada = decodificarPopulacao(populacao, dominio_max, dominio_min, precisao_dominio)
    aptidoes = calculandoAptidao(populacao_decodificada)
    aptidoes_relativas = particaoPopulacao(aptidoes)
    print(f'Geração {contador}: Melhor aptidão = {max(aptidoes)}')

TypeError: int() can't convert non-string with explicit base

In [301]:
t = 0
individuos = 100
NGEN = 100
taxa_mutacao = 0.01  #Taxa de mutação de 1%
taxa_de_cruzamento = 75 ## 75% 
precisao_casas_decimais = 5 ## Precisão de casas decimais
dominio_max = 100
dominio_min = -100

In [302]:
bits = bits_necessarios(dominio_max, dominio_min, precisao_casas_decimais)
bits

25

In [303]:
precisao_dominio = precisao(dominio_max, dominio_min, bits)
precisao_dominio

5.9604646551747515e-06

In [304]:
populacao = gerarPopulacao(individuos)
populacao[:2]

['11101011011110010000010111000001001000000010000011',
 '10111010100110000111000000110000001011001101110100']

In [305]:
populacao_decodificada = decodificarPopulacao(populacao, dominio_max, dominio_min, precisao_casas_decimais)
populacao_decodificada

[[0.9198154, 0.508793],
 [0.7288885500000001, 0.37636915000000004],
 [0.1510046, 0.7982940500000001],
 [0.47171055000000006, 0.328082],
 [0.7225478, 0.48312605],
 [0.9718513999999999, 0.64653985],
 [0.8898179500000001, 0.18134540000000002],
 [0.2380455, 0.48712355],
 [0.34319825000000004, 0.6712089],
 [0.4669108, 0.7593003500000001],
 [0.9364840000000001, 0.8791702000000001],
 [0.3674915, 0.10746839999999999],
 [0.3920216, 0.2361714],
 [0.33608225, 0.6585384500000001],
 [0.83595495, 0.45275385],
 [0.05058959999999999, 0.9112818],
 [0.2327688, 0.6763541000000001],
 [0.9736792, 0.24117945],
 [0.5000083, 0.324506],
 [0.1121089, 0.12011145],
 [0.22604500000000002, 0.98913445],
 [0.25911025, 0.89117515],
 [0.18009609999999998, 0.63520955],
 [0.03551755, 0.8139512],
 [0.6786141000000001, 0.68633415],
 [0.30417385, 0.59048015],
 [0.5946596000000001, 0.50177575],
 [0.14880895000000002, 0.48413280000000003],
 [0.07508865, 0.054804099999999994],
 [0.6428165, 0.4596383],
 [0.8914925499999999, 0.4

In [306]:
aptidoes = calcular_aptidoes(populacao_decodificada)
aptidoes_relativas = particaoPopulacao(aptidoes)

In [307]:
aptidoes

[0.1076949413,
 0.8351293459,
 0.7828049705,
 0.3845356865,
 0.2848665717,
 0.7172445146,
 0.8544460856,
 0.4762805906,
 0.9475396889,
 0.1999747545,
 0.7792812887,
 0.685719418,
 0.0633657388,
 0.0606653361,
 0.470105505,
 0.9038102302,
 0.551938007,
 0.8732620057,
 0.9599942085,
 0.5626492425,
 0.3808207675,
 0.0906114605,
 0.9566930866,
 0.9018943178,
 0.4283758058,
 0.7861137963,
 0.5470058124,
 0.8424280574,
 0.9827049852,
 0.7509419224,
 0.3882245498,
 0.0149313357,
 0.7039977952,
 0.0825514415,
 0.0586158907,
 0.873989327,
 0.0725808986,
 0.1197475941,
 0.0855495764,
 0.1628081245,
 0.3358765173,
 0.1450657767,
 0.5895983274,
 0.5277382716,
 0.1768460252,
 0.4670085147,
 0.3415018789,
 0.3083751139,
 0.8400091936,
 0.0483626031,
 0.851605568,
 0.8977772122,
 0.2619949801,
 0.2909060762,
 0.9253620363,
 0.9984039737,
 0.3147890098,
 0.8545645903,
 0.4719849334,
 0.9240175712,
 0.7418148383,
 0.5568363294,
 0.3912454642,
 0.8645207449,
 0.6260927523,
 0.6073284075,
 0.2606997621,


In [308]:
aptidoes_relativas

[0.0020615675159522274,
 0.015986596123673927,
 0.014984968458390733,
 0.007361035443665629,
 0.005453103586002847,
 0.013729967020227003,
 0.016356369880908196,
 0.009117276839627209,
 0.018138428965481214,
 0.0038280484943049174,
 0.014917515819968318,
 0.01312649285232924,
 0.0012129887175526383,
 0.0011612958300399202,
 0.008999069282916423,
 0.017301330858013334,
 0.010565560947574545,
 0.016716556619418566,
 0.018376841584720464,
 0.010770602474093435,
 0.007289922016773462,
 0.0017345442718560483,
 0.018313649334526644,
 0.017264655200646593,
 0.008200251889242078,
 0.015048308181713694,
 0.010471145629204202,
 0.01612631286760893,
 0.018811586234205358,
 0.014375025000237018,
 0.007431650095182861,
 0.0002858254647555837,
 0.013476389590513746,
 0.0015802540782055342,
 0.00112206399601544,
 0.01673047948290352,
 0.0013893913774052308,
 0.0022922873361554317,
 0.0016376463516370686,
 0.003116580377413753,
 0.006429569570106134,
 0.002776944667141014,
 0.011286479611346479,
 0.01

In [309]:
sum(aptidoes_relativas)

0.9999999999999996

In [310]:
sum(aptidoes)

52.23934722810001